In [1]:
from bs4 import BeautifulSoup, NavigableString
from urllib.request import urlopen
import re
import time

In [2]:
soup_url = 'https://press.rebus.community/aalh/'

In [3]:
def get_text_links(soup):
    '''
    get a list of urls that lead to the texts
    
    soup: (BeautifulSoup) an html parsed bs object
    
    return: (list) a list of urls
    '''
    tag_list = soup.findAll("li", {"id" : re.compile(r'toc-chapter-.*')})
    url_list = []
    is_text = False
    for tag in tag_list:
        if tag.find('a').contents[0] == 'Introducción':
            is_text = True
            continue
        elif is_text == True:
            url_list.append(tag.find('a').attrs['href'])
            is_text = False
            
    return url_list

In [4]:
soup = BeautifulSoup(urlopen(soup_url), 'html.parser')
url_list = get_text_links(soup)

print(url_list)

['https://press.rebus.community/aalh/chapter/lo-que-sucedio-a-un-mancebo-el-dia-que-se-caso/', 'https://press.rebus.community/aalh/chapter/carta-a-luis-de-santangel/', 'https://press.rebus.community/aalh/chapter/tercera-carta-relacion-de-hernan-cortes-al-emperador/', 'https://press.rebus.community/aalh/chapter/romance-del-enamorado-y-la-muerte/', 'https://press.rebus.community/aalh/chapter/la-vida-de-lazarillo-de-tormes-y-de-sus-fortunas-y-adversidades/', 'https://press.rebus.community/aalh/chapter/historia-de-la-monja-alferez/', 'https://press.rebus.community/aalh/chapter/a-su-retrato/', 'https://press.rebus.community/aalh/chapter/viaje-a-la-habana-carta-1/', 'https://press.rebus.community/aalh/chapter/la-flor-de-la-cana/', 'https://press.rebus.community/aalh/chapter/al-partir/', 'https://press.rebus.community/aalh/chapter/emancipacion-moral-de-la-mujer/', 'https://press.rebus.community/aalh/chapter/la-rosa-de-pasion/', 'https://press.rebus.community/aalh/chapter/las-medias-rojas/', '

In [5]:
def get_text(url):
    '''
    get a dictionary of the content and metadata of a text scraped from the url given
    
    url: (str) an url lead to a text
    
    return: (dict) a dictionary with 3 keys: 'author', 'title', and 'content'
    '''
    soup = BeautifulSoup(urlopen(url), 'html.parser')
    text_dict = {}
    
    text_dict['title'] = soup.find('h1', {'class':'entry-title'}).contents[0].strip()
    for child in soup.find('h2').contents:
        if isinstance(child, NavigableString):
            text_dict['author'] = child.strip()
            break
    
    content_section = soup.find('section', {'data-type':'chapter'})
    content = ''
    for tag in content_section.contents:
        if tag.name == 'p':
            paragraph = []
            for child in tag.contents:
                if isinstance(child, NavigableString):
                    paragraph.append(child)
            content += ''.join(paragraph) + '\n\n'
    
    text_dict['content'] = content.strip()
    return text_dict

In [7]:
aalh_corpus = []
for url in url_list:
    aalh_corpus.append(get_text(url))
    time.sleep(1)

In [8]:
aalh_corpus[0]

{'title': 'Lo que sucedió a un mancebo el día que se casó',
 'author': 'Don Juan Manuel',
 'content': 'Un día hablaba el Conde Lucanor con Patronio, su consejero, y le decía:\n\n—Patronio, un pariente mío me ha dicho que lo quieren casar con una mujer muy rica, y aunque es más honrada que él, el casamiento sería muy bueno para él si no fuera por un embargo que ahí hay, y el embargo es éste: \xa0Me dijo que le dijeron otros que aquella mujer era la más fuerte y la más brava cosa del mundo, y ahora ruego a vos que me aconsejéis si le mande que case con aquella mujer—pues sabe de cual manera es—, o si le mande que lo no haga.\n\n—Señor conde Lucanor —dijo Patronio— si él fuera tal como fue un hijo de un hombre bueno que era moro, aconsejadle que case con ella; más si no fuere tal, no se lo aconseja. Y el conde le rogó que le dijera cómo era aquello.\n\nPatronio le dijo que en una villa vivía un moro honrado que vivía con un hijo, el mejor mancebo que en el mundo podría ser, pero no era ta

In [9]:
len(aalh_corpus)

21